# UES workload analysis

In [1]:
import json
import re
import warnings

import numpy as np
import pandas as pd

from transform import db, mosp

In [2]:
df = pd.read_csv("workloads/job-ues-results-fks-nonlj.csv")
df = df[df.run == 1].copy()
df.result = df.result.apply(json.loads)
df

,label,query,result,rt_total,run,workload
0,1a,select count(*) from movie_companies as mc j...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.443611,1,ues
1,1b,select count(*) from movie_info_idx as mi_idx...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.322460,1,ues
2,1c,select count(*) from movie_companies as mc j...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.435026,1,ues
3,1d,select count(*) from movie_info_idx as mi_idx...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.330217,1,ues
4,2a,select count(*) from movie_keyword as mk joi...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.843378,1,ues
...,...,...,...,...,...,...
447,32a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.435622,1,transformed
448,32b,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.427622,1,transformed
449,33a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",1.055222,1,transformed
450,33b,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",1.284310,1,transformed


In [3]:
def parse_query_plans(sample: pd.Series) -> db.PlanNode:
    query, plan, workload = sample["query"], sample["result"], sample["workload"]
    if workload == "ues":
        parsed_plan = db.parse_explain_analyze(query, plan)
    elif workload == "transformed":
        parsed_plan = db.parse_explain_analyze(query, plan, with_subqueries=False)
    else:
        warnings.warn("Unknown workload '{}', assuming no subqueries".format(workload))
        parsed_plan = db.parse_explain_analyze(query, plan, with_subqueries=False)
    return parsed_plan

In [4]:
df["query"] = df["query"].apply(mosp.MospQuery.parse)
df.result = df.apply(parse_query_plans, axis="columns")
df["subquery"] = df.result.apply(db.PlanNode.extract_subquery)

/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Aggregate
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Gather
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Hash
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Sort
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Materialize
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/transform/db.py:331: UserWarning: Unknown node type: Gather Merge
  warnings.warn("Unknown node type: {}".format(node_t

In [11]:
df_ues = df[df.workload == "ues"].copy()
df_trans = df[df.workload == "transformed"].copy()

In [10]:
df_sqs = df_ues[~df_ues.subquery.isna()]
len(df_sqs)

51

In [19]:
df_cmp = pd.merge(
    df_sqs.drop(columns=["run", "workload"]),
    df_trans.drop(columns=["run", "workload", "subquery"]),
    on="label", how="inner",
    suffixes=("_orig", "_trans"))
len(df_cmp)

51

In [24]:
def calculate_subquery_rows(parse_tree, *, target_predicate=""):
    if not target_predicate and parse_tree.is_subquery():
        parent = parse_tree.parent if parse_tree.parent else parse_tree
        left, right = parent.left, parent.right
        return parent, left.proc_rows + right.proc_rows
    elif target_predicate and db.compare_predicate_strs(parse_tree.join_pred, target_predicate):
        parent = parse_tree.parent if parse_tree.parent else parse_tree
        left, right = parent.left, parent.right
        return parent, left.proc_rows + right.proc_rows
    elif target_predicate:
        print(f"{target_predicate} does not match {parse_tree.join_pred}")

    if not parse_tree.children:
        return None
    child_rows = [calculate_subquery_rows(child) for child in parse_tree.children]
    filtered = [c for c in child_rows if c]
    return db._simplify_plan_tree(filtered) if filtered else None

In [26]:
df_cmp.result_orig.apply(calculate_subquery_rows)

0     (Hash Join (mi_idx.movie_id = t.id) <- [Hash J...
1     (Hash Join (mi_idx.movie_id = t.id) <- [Hash J...
2     (Hash Join (mc.movie_id = t.id) <- [Hash Join ...
3     (Hash Join (t.id = ml.linked_movie_id) <- [Has...
4     (Hash Join (t.id = ml.linked_movie_id) <- [Has...
5     (Hash Join (ci.person_id = a1.person_id) <- [H...
6     (Hash Join (ci.person_id = an1.person_id) <- [...
7     (Hash Join (mk.movie_id = ml.movie_id) <- [Has...
8     (Hash Join (mc.movie_id = t.id) <- [Hash Join ...
9     (Hash Join (mi.movie_id = t.id) <- [Hash Join ...
10    (Hash Join (mc.movie_id = t.id) <- [Hash Join ...
11    [(Hash Join (mi.movie_id = miidx.movie_id) <- ...
12    [(Hash Join (mi.movie_id = miidx.movie_id) <- ...
13    [(Hash Join (mi.movie_id = miidx.movie_id) <- ...
14    [(Hash Join (mi.movie_id = miidx.movie_id) <- ...
15    (Hash Join (mi_idx.movie_id = t.id) <- [Hash J...
16    (Hash Join (mi_idx.movie_id = t.id) <- [Hash J...
17    (Hash Join (mi_idx.movie_id = t.id) <- [Ha

In [ ]:
calculate_subquery_rows(eap)

In [ ]:
df_trans = pd.read_csv("workloads/job-ues-results-flattened-nofk.csv")
df_trans = df_trans[df_trans.run == 1].copy()
df_trans.flattened_query_result = df_trans.flattened_query_result.apply(json.loads)
df_trans

In [ ]:
p_trans = df_trans.iloc[0].flattened_query_result
p_trans

In [ ]:
q_trans = mosp.MospQuery.parse(df_trans.iloc[0]["query"])
q_trans

In [ ]:
eap_trans = db.parse_explain_analyze(q_trans, p_trans, with_subqueries=False)
print(eap_trans.pretty_print())

In [ ]:
print(eap.pretty_print())

In [ ]:
eap_trans.left

In [ ]:
eap_trans.right

In [ ]:
eap_trans.left.proc_rows + eap_trans.right.proc_rows

In [ ]:
calculate_subquery_rows(eap)

In [ ]:
calculate_subquery_rows(eap_trans, target_predicate="(mi_idx.info_type_id = it.id)")